<a href="https://colab.research.google.com/github/cvillarr123/appinventorGoogle/blob/main/notebooks/pruebaflask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn

In [ ]:
!pip install flask-ngrok

In [1]:
import PIL.Image as Image
import io
import base64
#from data import byte_data

In [2]:
from google.colab import drive
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path='/content/drive/My Drive/uploadImage/'

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,jsonify, render_template,request
app = Flask(__name__)
run_with_ngrok(app) #starts ngrok when the app is running

@app.route('/', methods=['GET','POST'])
def home():
    if request.method=='GET':

        result = {'result':'voila'}
        return result
    if request.method=='POST':
        imageID = request.form.get('filename')
        Byte_Im = request.form.get('file')
        imagenpath = path + imageID
        #*************
        image2bytes = Byte_Im.encode('utf-8')
        #image2bytes = Byte_Im
        with open(imagenpath,'wb') as file_to_save:
            decoded_image_data = base64.decodebytes(image2bytes)
            file_to_save.write(decoded_image_data)
        #************

        #c= base64.b64decode(CadenaImagen)
        #im= Image.open(io.BytesIO(c))
        #im.show()
        result = {'result':'Success'}
        return result
    else:
      result = {'result':'NOPOST'}
      return result 

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e526-34-86-248-148.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [19/Jan/2022 04:43:01] "POST / HTTP/1.1" 200 -
